<a href="https://colab.research.google.com/github/annnanya/Fakenews/blob/main/FakeNewsDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fake News Detection**

Fake news also referred to hoax news occupies large sphere of cyber space today world-wide. Often sensational news are created and spread through social media to achieve intended end. The problem is to identify the authenticity of the news and online content.

In [ ]:
!unzip "/content/drive/MyDrive/fakenewskdd2020.zip"

Archive:  /content/drive/MyDrive/fakenewskdd2020.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
# Load the data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Load the train and test files
train = pd.read_csv("/content/train.csv", sep='\t', encoding='utf-8')
test = pd.read_csv("/content/test.csv", sep='\t', encoding='utf-8')

In [ ]:
train.head()

,text,label
0,Get the latest from TODAY Sign up for our news...,1
1,2d Conan On The Funeral Trump Will Be Invited...,1
2,It’s safe to say that Instagram Stories has fa...,0
3,Much like a certain Amazon goddess with a lass...,0
4,At a time when the perfect outfit is just one ...,0


# **Bag of Words Pipeline** 

In [ ]:
import nltk 
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
train['text'].loc[0]

'Get the latest from TODAY Sign up for our newsletter  No one ever truly gets over losing a loved one, and Blake Shelton is no exception. He was just 14 when his older brother Richie died on Nov. 13, 1990. And, as Shelton noted in a tweet Monday, "It changed my life forever."  Richie was 24 when he died in a car accident in the Sheltons\' home state of Oklahoma. Two years ago, Shelton sent out a message for the 25th anniversary of his loss:  Richie, who was Blake\'s half-brother (they shared a mother), was a passenger in a car that collided with a school bus in Ada, south of Oklahoma City.  Richie, driver Redena McManus and a 3-year-old boy, Christopher McManus, all died during or shortly after the collision, while the bus driver and passengers were uninjured, according to police reports.  The accident has clearly remained with Blake, who told 60 Minutes in 2014, "I remember picking up the phone to call him a week after he was dead, to tell him something. I was picking up the phone to 

In [ ]:
train.text.loc[ : 5].values.tolist()[0]

'Get the latest from TODAY Sign up for our newsletter  No one ever truly gets over losing a loved one, and Blake Shelton is no exception. He was just 14 when his older brother Richie died on Nov. 13, 1990. And, as Shelton noted in a tweet Monday, "It changed my life forever."  Richie was 24 when he died in a car accident in the Sheltons\' home state of Oklahoma. Two years ago, Shelton sent out a message for the 25th anniversary of his loss:  Richie, who was Blake\'s half-brother (they shared a mother), was a passenger in a car that collided with a school bus in Ada, south of Oklahoma City.  Richie, driver Redena McManus and a 3-year-old boy, Christopher McManus, all died during or shortly after the collision, while the bus driver and passengers were uninjured, according to police reports.  The accident has clearly remained with Blake, who told 60 Minutes in 2014, "I remember picking up the phone to call him a week after he was dead, to tell him something. I was picking up the phone to 

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

# create a function to tokenize the data
def preprocess_data(data):
  
  # 1. Tokenization
  tk = RegexpTokenizer('\s+', gaps = True)
  text_data = [] # List for storing the tokenized data
  for values in data.text:
    tokenized_data = tk.tokenize(values) # Tokenize the news
    text_data.append(tokenized_data) # append the tokenized data

  # 2. Stopword Removal

  # Extract the stopwords
  sw = stopwords.words('english')
  clean_data = [] # List for storing the clean text
  # Remove the stopwords using stopwords
  for data in text_data:
    clean_text = [words.lower() for words in data if words.lower() not in sw]
    clean_data.append(clean_text) # Appned the clean_text in the clean_data list
  
  # 3. Stemming

  # Create a stemmer object
  ps = PorterStemmer()
  stemmed_data = [] # List for storing the stemmed data
  for data in clean_data:
    stemmed_text = [ps.stem(words) for words in data] # Stem the words
    stemmed_data.append(stemmed_text) # Append the stemmed text
  

 
  # Flatten the stemmed data

  updated_data = []
  for data in stemmed_data:
    updated_data.append(" ".join(data))

  # TFID Vector object
  tfidf = TfidfVectorizer()
  tfidf_matrix = tfidf.fit_transform(updated_data)

  return tfidf_matrix

In [ ]:
# Call the above function on the merged data
train_len = train.shape[0]
merged_data = pd.concat((train.drop('label', axis=1), test.drop('id', axis=1)), axis=0).reset_index().drop('index', axis=1)

In [ ]:
# preprocess the merged data
preprocessed_data = preprocess_data(merged_data)

In [ ]:
train_data = preprocessed_data[ : train_len]
test_data = preprocessed_data[train_len : ]

# ***Modelling***

In [ ]:
# Model selection
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(train_data, train.label, test_size=0.2, random_state = 42)

In [ ]:
# Metrics
from sklearn.metrics import accuracy_score

# model
def compute_metrics(data, y_true, model_obj, model):

  # Make predictions
  y_pred = model_obj.predict(data)

  # Compute accuracy
  acc = accuracy_score(y_true = y_true, y_pred = y_pred)

  # Make DataFrame
  metrics = pd.DataFrame(data = np.array([acc]), index=[model], columns=['Accuracy Score'])
  return metrics

#1. LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

# Model object
lr_reg = LogisticRegressionCV(Cs=20, cv=3, random_state=42)

# fit the model
lr_reg.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


LogisticRegressionCV(Cs=20, class_weight=None, cv=3, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='auto', n_jobs=None,
                     penalty='l2', random_state=42, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)

In [ ]:
# Compute the Logistic Regression Metrics
lr_metrics =  compute_metrics(X_test, y_test, lr_reg, 'LogisticRegression')

In [ ]:
lr_metrics_train =  compute_metrics(X_train, y_train, lr_reg, 'LogisticRegression')

In [ ]:
lr_metrics

,Accuracy Score
LogisticRegression,0.778557


#2. Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Model Object
mnb = MultinomialNB(alpha=0.0)

# Fit the object
mnb.fit(X_train, y_train)

# Compute metrics
mnb_metrics = compute_metrics(X_test, y_test, mnb, 'Naive Bayes')
mnb_metrics

/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


,Accuracy Score
Naive Bayes,0.719439


# 3. DecisionTree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Model Object
dt_clf = DecisionTreeClassifier()

# Fit the object
dt_clf.fit(X_train, y_train)

dt_metrics = compute_metrics(X_test, y_test, dt_clf, "DecisionTree")

dt_metrics

,Accuracy Score
DecisionTree,0.684369


In [ ]:
# Concatenate all the metrics
model_metrics = pd.concat((lr_metrics, mnb_metrics, dt_metrics), axis=0).sort_values(by='Accuracy Score', ascending=False)


In [ ]:
model_metrics

,Accuracy Score
LogisticRegression,0.778557
Naive Bayes,0.719439
DecisionTree,0.684369


# ***Making Prediction***

# Logistic Regression Prediction

In [ ]:
# make predictions--> Logistic Regression
predictions2 = lr_reg.predict(test_data)
# Submissions
test_ID = test.id
submissions2 = pd.DataFrame({'id' : test_ID, 'label' : predictions2})
submissions2.to_csv("/content/predictions_5.csv", index=False) # Convert the submissions to .csv